In [1]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [2]:
#simple neural network to classify digits using MNIST dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [3]:
sess = tf.InteractiveSession()

In [4]:
#Placeholder 'X': represents the "space" allocated input or the images.
#Each input has 784 pixels distributed by a 28 width x 28 height matrix
#The 'shape' argument defines the tensor size by its dimensions.
#1st dimension = None. Indicates that the batch size, can be of any size.
#2nd dimension = 784. Indicates the number of pixels on a single flattened MNIST image.
x  = tf.placeholder(tf.float32, shape=[None, 784])

In [5]:
#Placeholder 'Y': represents the final output or the labels.

#10 possible classes (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
#The 'shape' argument defines the tensor size by its dimensions.
#1st dimension = None. Indicates that the batch size, can be of any size.
#2nd dimension = 10. Indicates the number of targets/outcomes
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [6]:
# Weight tensor
W = tf.Variable(tf.zeros([784, 10],tf.float32))
# Bias tensor
b = tf.Variable(tf.zeros([10],tf.float32))

In [7]:
# run the op initialize_all_variables using an interactive session
sess.run(tf.global_variables_initializer())

In [8]:
# mathematical operation to add weights and biases to the inputs
tf.matmul(x,W) + b

<tf.Tensor 'add:0' shape=(?, 10) dtype=float32>

In [9]:
#Softmax is an activation function that is normally used in classification problems. It generate the probabilities for 
#the output. For example, our model will not be 100% sure that one digit is the number nine, instead, the answer will
#be a distribution of probabilities where, if the model is right, the nine number will have a larger probability than 
#the other other digits.
y = tf.nn.softmax(tf.matmul(x,W) + b)

In [10]:
#cost function - It is a function that is used to minimize the difference between the right answers (labels) and 
# estimated outputs by our Network.
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [11]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [12]:
#training batches
#Load 50 training examples for each training iteration   
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [13]:
#test
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
acc = accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}) * 100
print("The final accuracy for the simple ANN model is: {} % ".format(acc) )

The final accuracy for the simple ANN model is: 91.72999858856201 % 


In [14]:
sess.close() #finish the session

In [16]:
#part2 - Simple Deep Neural Network with Dropout (more than 1 hidden layer)

# Architecture of our network is:
    
# 1- (Input) -> [batch_size, 28, 28, 1]  >> Apply 32 filter of [5x5]
# 2- (Convolutional layer 1)  -> [batch_size, 28, 28, 32]
# 3- (ReLU 1)  -> [?, 28, 28, 32]
# 4- (Max pooling 1) -> [?, 14, 14, 32]
# 5- (Convolutional layer 2)  -> [?, 14, 14, 64] 
# 6- (ReLU 2)  -> [?, 14, 14, 64] 
# 7-  (Max pooling 2)  -> [?, 7, 7, 64] 
# 8- [fully connected layer 3] -> [1x1024]
# 9- [ReLU 3]  -> [1x1024]
#10- [Drop out]  -> [1x1024]
#11- [fully connected layer 4] -> [1x10]


In [17]:
import tensorflow as tf

# finish possible remaining session
sess.close()

#Start interactive session
sess = tf.InteractiveSession()

In [18]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [19]:
width = 28 # width of the image in pixels 
height = 28 # height of the image in pixels
flat = width * height # number of pixels in one image 
class_output = 10 # number of possible classifications for the problem

In [20]:
x  = tf.placeholder(tf.float32, shape=[None, flat])
y_ = tf.placeholder(tf.float32, shape=[None, class_output])

In [21]:
#Converting images of the data set to tensors
#The input image is 28 pixels by 28 pixels, 1 channel (grayscale). In this case, the first dimension is the batch number
#of the image, and can be of any size (so we set it to -1). The second and third dimensions are width and height, and the 
#last one is the image channels.
x_image = tf.reshape(x, [-1,28,28,1])  
x_image

<tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>

In [22]:
#We define a kernel here. The Size of the filter/kernel is 5x5; Input channels is 1 (grayscale); and we need 32 different
#feature maps (here, 32 feature maps means 32 different filters are applied on each image. So, the output of convolution 
#layer would be 28x28x32). In this step, we create a filter / kernel tensor of shape [filter_height, filter_width,
#in_channels, out_channels]
W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32])) # need 32 biases for 32 outputs

In [23]:
convolve1= tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1

In [24]:
#In this step, we just go through all outputs convolution layer, convolve1, and wherever a negative number occurs,
#we swap it out for a 0. It is called ReLU activation Function.
#Let f(x) is a ReLU activation function  𝑓(𝑥)=𝑚𝑎𝑥(0,𝑥)
h_conv1 = tf.nn.relu(convolve1)

In [25]:
conv1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #max_pool_2x2
conv1


##first layer completed

<tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>

In [28]:
# second layer
#here, the input image is [14x14x32], the filter is [5x5x32], we use 64 filters of size [5x5x32], and the output of 
#the convolutional layer would be 64 convolved image, [14x14x64].
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64])) #need 64 biases for 64 outputs

In [29]:
convolve2= tf.nn.conv2d(conv1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2

In [30]:
h_conv2 = tf.nn.relu(convolve2)

In [31]:
conv2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #max_pool_2x2
conv2


##second layer completed

<tf.Tensor 'MaxPool_1:0' shape=(?, 7, 7, 64) dtype=float32>

In [32]:
#Now we need a fully connected layer to use the Softmax and create the probabilities in the end. Fully connected layers take
#the high-level filtered images from previous layer, that is all 64 matrices, and convert them to a flat array
layer2_matrix = tf.reshape(conv2, [-1, 7 * 7 * 64])

In [33]:
W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024])) # need 1024 biases for 1024 outputs

In [34]:
fcl = tf.matmul(layer2_matrix, W_fc1) + b_fc1

In [35]:
h_fc1 = tf.nn.relu(fcl)
h_fc1

##third layer completed

<tf.Tensor 'Relu_2:0' shape=(?, 1024) dtype=float32>

In [36]:
#It is a phase where the network "forget" some features. At each training step in a mini-batch, some units get switched off randomly
#so that it will not interact with the network. That is, it weights cannot be updated, nor affect the learning of the other
#network nodes. This can be very useful for very large neural networks to prevent overfitting.
keep_prob = tf.placeholder(tf.float32)
layer_drop = tf.nn.dropout(h_fc1, keep_prob)
layer_drop

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


<tf.Tensor 'dropout/mul_1:0' shape=(?, 1024) dtype=float32>

In [37]:
W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1)) #1024 neurons
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10])) # 10 possibilities for digits [0,1,2,3,4,5,6,7,8,9]

In [38]:
fc=tf.matmul(layer_drop, W_fc2) + b_fc2

In [40]:
y_CNN= tf.nn.softmax(fc)
y_CNN

<tf.Tensor 'Softmax_1:0' shape=(?, 10) dtype=float32>

In [41]:
import numpy as np
layer4_test =[[0.9, 0.1, 0.1],[0.9, 0.1, 0.1]]
y_test=[[1.0, 0.0, 0.0],[1.0, 0.0, 0.0]]
np.mean( -np.sum(y_test * np.log(layer4_test),1))

0.10536051565782628

In [42]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_CNN), reduction_indices=[1]))

In [43]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [44]:
correct_prediction = tf.equal(tf.argmax(y_CNN, 1), tf.argmax(y_, 1))

In [45]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [46]:
sess.run(tf.global_variables_initializer())

In [47]:
for i in range(1100):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, float(train_accuracy)))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.08
step 100, training accuracy 0.88
step 200, training accuracy 0.82
step 300, training accuracy 0.94
step 400, training accuracy 0.92
step 500, training accuracy 0.96
step 600, training accuracy 0.96
step 700, training accuracy 0.88
step 800, training accuracy 0.92
step 900, training accuracy 1
step 1000, training accuracy 0.96


In [48]:
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.98
step 100, training accuracy 0.96
step 200, training accuracy 0.94
step 300, training accuracy 0.98
step 400, training accuracy 1
step 500, training accuracy 0.98
step 600, training accuracy 0.98
step 700, training accuracy 1
step 800, training accuracy 0.94
step 900, training accuracy 1
step 1000, training accuracy 0.92
step 1100, training accuracy 0.98
step 1200, training accuracy 0.96
step 1300, training accuracy 1
step 1400, training accuracy 1
step 1500, training accuracy 0.94
step 1600, training accuracy 0.98
step 1700, training accuracy 1
step 1800, training accuracy 1
step 1900, training accuracy 0.98
step 2000, training accuracy 0.98
step 2100, training accuracy 0.98
step 2200, training accuracy 1
step 2300, training accuracy 0.96
step 2400, training accuracy 1
step 2500, training accuracy 0.98
step 2600, training accuracy 0.98
step 2700, training accuracy 0.98
step 2800, training accuracy 1
step 2900, training accuracy 1
step 3000, training accur

In [49]:
#Evaluate the model
# evaluate in batches to avoid out-of-memory issues
n_batches = mnist.test.images.shape[0] // 50
cumulative_accuracy = 0.0
for index in range(n_batches):
    batch = mnist.test.next_batch(50)
    cumulative_accuracy += accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
print("test accuracy {}".format(cumulative_accuracy / n_batches))

test accuracy 0.9925000044703484


In [58]:
sess.close() #finish the session